In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [2]:
# Program bitstream to FPGA
overlay = Overlay('/home/ubuntu/workspace/hello_dma.bit')

In [3]:
# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [4]:
# Maximum data that can be sent by AXI DMA for 1 transaction is 67108863 bytes
# floor(67108863 bytes/8) = 8388607 word (64-bit)
# We divide by 8 because we use uint64 data type
data_size = 8388607

In [5]:
# Allocate physical memory for AXI DMA MM2S
input_buffer = allocate(shape=(data_size,), dtype=np.uint64)

In [6]:
# Write data to physical memory
for i in range(data_size):
    input_buffer[i] = i + 0xcafe000000000000

In [7]:
# Check the written data
for i in range(10):
    print(hex(input_buffer[i]))

0xcafe000000000000
0xcafe000000000001
0xcafe000000000002
0xcafe000000000003
0xcafe000000000004
0xcafe000000000005
0xcafe000000000006
0xcafe000000000007
0xcafe000000000008
0xcafe000000000009


In [8]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)

In [9]:
# Allocate physical memory for AXI DMA S2MM
output_buffer = allocate(shape=(data_size,), dtype=np.uint64)

In [10]:
# Check the memory content
for i in range(10):
    print(hex(output_buffer[i]))

0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0


In [11]:
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [12]:
# Check the memory content after DMA transfer
for i in range(10):
    print(hex(output_buffer[i]))

0xcafe000000000000
0xcafe000000000001
0xcafe000000000002
0xcafe000000000003
0xcafe000000000004
0xcafe000000000005
0xcafe000000000006
0xcafe000000000007
0xcafe000000000008
0xcafe000000000009


In [13]:
# Compare arrays
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

Arrays are equal: True


In [14]:
# Get information about physical address
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

Input buffer address   : 0x36b00000
Output buffer address  : 0x3ab00000
DMA Source address     : 0x36b00000
DMA Destination address: 0x3ab00000


In [15]:
# Delete buffer to prevent memory leak
del input_buffer, output_buffer